<div class="alert alert-block alert-info">
    
These blue boxes contain instructions for you to follow, or stuff for you to do
<h2>How to access this Jupyter notebook</h2>

* <b>Step 1</b>: Open a web browser,  and go to [this page](https://jupyter.warwick.ac.uk/module/CH274), 
* <b>Step 2</b>: Enter your SCRTP username and password and press the "Start Server" button.<br>
* <b>Step 3</b>: Wait (it could take a few minutes) until the blue box says "Jupyter notebook server running!". At that point, click on the weblink below said message.<br>
* <b>Step 4</b>: Select the Jupyter Notebook you want to work on. <i>Remember to make a copy of the orginal notebook</i> (which is read-only). To do so, in the toolbar on top of the notebook, select File and then Make a Copy <br>
* <b>Step 5</b>: You're all set! <br>
* <b>Step 6</b>: <font color="red">When you are done, remember to click the "Stop Server" button in the launcher web browser tab.</font> Please do, it's really quite important. <br>
<b> Remember: </b> You can access your copy of the Notebook at any time from any device off and on campus by going through the same steps on e.g. your laptop - all the changes you have made will be saved and synced! <br>

<div/>

***

# The particle in a box in 1D (pib1d)
Here's a quick reminder of the particle in a box 


<div class="col-md-12">
                    </div>
                    <div class="col-md-12">    
                    <div class="col-md-4">
                        <img src="tik1d.png" alt="Motivation">
                    </div>    
                    <div class="col-md-8">
                        <p>  Schrödinger equation (SE): $\hat{H}\psi = E\psi $</p>
                        <p class="">  
                        SE: action of energy operator $\hat{H}$ defines the energy and wave function $\psi$  </p>
                        <p class=""> $$ \hat{H} = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2} +V  $$</p>
                        <p class=" margin-top-10"> boundary conditions: $ \psi(0) = 0 \quad\text{and}\quad \psi(L) = 0 $</p>
                    </div>
                    </div>                    
                    <div class="col-md-6">                   
                        <p><b>wave function</b></p>
                        <p> $$ \psi_n(x) = \sqrt{\frac{2}{L}}\sin{\frac{n\pi x}{L}}  $$ </p>
                    </div>
                    <div class="col-md-6">                    
                        <p><b>energies</b></p>
                        <p> $$ E_n = \frac{\hbar^2\pi^2}{2m}\frac{n^2}{L^2}  $$ </p>
                    </div>

In [ ]:
#Let's import some important packages before we start.
import numpy
#it's nice to abbreviate packages which we use a lot, so
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp  #scipy is now called sp
%matplotlib notebook 
%matplotlib inline 

In [ ]:
def make_xgrid(start, stop, N):
    x = np.linspace(start,stop,N)
    return x

def potential(x, box_l=0.0, box_r=5.0, height=100000.0): 
    pot = np.zeros_like(x)
    
    for i, xvalue in enumerate(x):
        if (xvalue<box_l): #left wall
            pot[i] = height
        elif (xvalue> box_l and xvalue < box_r): #inside the box
            pot[i] = 0.0
        elif (xvalue>box_r): #right wall
            pot[i] = height
        else:
            raise ValueError("COMPUTER SAYS NO. This point should never be reached.")
    return pot



In [ ]:
def create_H(x, pot):
    """
    This function creates a Hamiltonian.
    It takes the x axis grid and the potential grid and returns the Hamiltonian matrix H=T+V
    """
    hbar = 1
    m = 1
    #distance between grid points
    dx = x[1]-x[0]
    
    ###kinetic energy T
    import scipy.ndimage
    #This is the second derivative
    L = scipy.ndimage.laplace(np.eye(len(x)), mode='wrap')/(2.0*dx*dx) #np.eye is a diagonal unit matrix (I) with size 'x' by 'x'
    #Here we add all the constants
    T= -(1./2.)*((hbar**2)/m)*L
    
    #We build the hamiltonian by adding the potential energy to the matrix diagonal
    V = np.diag(pot)   #np.diag takes a list of numbers and puts it onto the diagonal; the offdiagonals are 0
    
    H = T+V
    return H
    

In [ ]:
def diagonalise_H(H):
    import scipy.linalg as la
    E, psi = la.eigh(H)
    return E, psi

In [ ]:
####PLEASE IGNORE THIS BOX, JUST EXECUTE IT
#### %matplotlib notebook 
%matplotlib inline 
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
    
def plot_pib1d(x, pot, E, psi, n=0):
        """
        plots the energies, the wave function and the probability density with interactive handles.
        """

        n=n-1
        import matplotlib.gridspec as gridspec
        plt.figure(figsize=(11, 8), dpi= 80, facecolor='w', edgecolor='k') # figsize determines the actual size of the figure
        gs1 = gridspec.GridSpec(2, 3)
        gs1.update(left=0.05, right=0.90, wspace=0.35,hspace=0.25)
        ax1 = plt.subplot(gs1[: ,0])
        ax2 = plt.subplot(gs1[0, 1:])
        ax3 = plt.subplot(gs1[1, 1:])


        ax1.plot(x, pot, color='gray')
        ax1.plot(np.ones(len(E)), E, lw=0.0, marker='_',ms=4000, color='blue')
        ax1.plot(1, E[n], lw=0.0, marker='_',ms=4000, color='red')
        ax1.set_ylim(-1,21)
        ax2.plot(x, psi[:,n])  
        ax3.plot(x, psi[:,n]*psi[:,n])

        #figure out boundaries
        start=0; stop=0
        for i, p in enumerate(pot):
            if (p>-0.001 and p<0.001):
                start=x[i-1]
                break
        for i, p in enumerate(pot):
            if (p>-0.001 and p<0.001):
                stop=x[i+1]

        ax2.axvline(start, color='gray')
        ax2.axvline(stop, color='gray')  
        ax3.axvline(start, color='gray')
        ax3.axvline(stop, color='gray')
        ax3.axhline(0.0,color='gray')

        #Labeling of x and y axes
        ax1.set_xlabel('x',fontsize=14)
        ax1.set_ylabel('Energy [eV]',fontsize=14)
        ax2.set_ylabel(r'wave function $\psi(x)$',fontsize=14)
        ax3.set_ylabel(r'density $|\psi|^2$(x)',fontsize=14)
        ax2.set_xlabel(r'x',fontsize=14)
        ax3.set_xlabel(r'x',fontsize=14)

        #Show the final result
        plt.show()
        return plot_pib1d


In [ ]:
def pib1d(N, box_l, box_r, height=100000.0):
    """
    calculates and visualises the energies and wave functions of the 1d particle in a box
    
    This program takes the number of grid points and the start and end point of the box as arguments and 
    returns a visualisation.
    """
    
    #make an x axis
    x = make_xgrid(box_l-1.0,box_r+1.0,N)
    #create a box potential on that axis
    pot = potential(x,box_l, box_r, height)
    #calculate Hamiltonian
    H = create_H(x,pot)
    #diagonalise Hamiltonian, solve for E and psi
    E, psi = diagonalise_H(H)
    #plot results
    interact(plot_pib1d,x=fixed(x),pot=fixed(pot),E=fixed(E),psi=fixed(psi),n=range(1,21))   
    
    return True

## Particle in a box
Use the next function to plot the particle in a box

In [ ]:
#Feel free to adjust these values

N = 2000 #Number of grid points
box_l = 0.0 #left box start
box_r = 3.0 #right box end
height = 10000.0 #height of box
pib1d(N,box_l,box_r, height)



## Harmonic Oscillator

In [ ]:
#potential function
def harmonic_potential(x, x0=2.5, k=10.):
    """
    Function calculates harmonic potential on x-axis centred around x0 with spring constant k
    """
    return 0.5*k*(x-x0)*(x-x0)


#new main program where we replaced the potential
def HO1D(N,box_l,box_r, k=10.0):
        
    x = make_xgrid(box_l-1.0,box_r+1.0,N)
    ######ONLY THE FOLLOWING LINES ARE DIFFERENT
    x0 = (box_l+box_r)/2  #define the centre of the harmonic oscillator in the middle of the box
    pot = harmonic_potential(x, x0, k)
    ######END OF DIFFERENCE##########
    H = create_H(x,pot)
    E, psi = diagonalise_H(H)
    interact(plot_pib1d,x=fixed(x),pot=fixed(pot),E=fixed(E),psi=fixed(psi),n=range(1,21))   
    
    return True

In [ ]:
#Number of grid points
N = 200
box_l = 0.0
box_r = 5.0
height = 10.0
HO1D(N,box_l,box_r, height)